In [1]:
import os.path
import imageio

import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import scipy.misc
import numpy as np

imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

LEARNING_RATE = 1e-4
KEEP_PROB = 0.5

# these need to be global for the video pipeline function
image_shape = (1, 1)
sess = tf.Session()
keep_prob = tf.placeholder(tf.float32)
logits = tf.placeholder(tf.float32, [None, None, None, 2])
input_image = tf.placeholder(tf.int32, [None, None, 3])

In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1
Default GPU Device: /gpu:0


In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    
    vgg_tag = 'vgg16'
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)

    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    graph = tf.get_default_graph()
    
    vgg_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    vgg_keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    vgg_layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    vgg_layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    vgg_layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)

    return vgg_input, vgg_keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out
            
#tests.test_load_vgg(load_vgg, tf)

In [4]:
def layers_(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output size 4096
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output size 512
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output size 256
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    conv_1x1_lay7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='same',kernel_initializer=tf.truncated_normal_initializer(stddev=1e-2),kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    conv_1x1_lay4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='same',kernel_initializer=tf.truncated_normal_initializer(stddev=1e-2),kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))

    output = tf.layers.conv2d_transpose(conv_1x1_lay7, num_classes, 4, 2, 'same',kernel_initializer=tf.truncated_normal_initializer(stddev=1e-2),kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))  # scale up by x2
    output = tf.add(output, conv_1x1_lay4)  # first skip layer

    conv_1x1_lay3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='same',kernel_initializer=tf.truncated_normal_initializer(stddev=1e-2),kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))

    output = tf.layers.conv2d_transpose(output, num_classes, 4, 2, 'same',kernel_initializer=tf.truncated_normal_initializer(stddev=1e-2),kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))  # scale up by x2
    output = tf.add(output, conv_1x1_lay3)  # second skip layer

    output = tf.layers.conv2d_transpose(output, num_classes, 16, 8, 'same',kernel_initializer=tf.truncated_normal_initializer(stddev=1e-2),kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))  # scale up by x8 to get original image size

    return output

In [5]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    layer7_1x1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 
                                padding='same',
                                name="layer7_1x1",
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01),
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))  
    layer4_1x1 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, 
                                name="layer4_1x1",
                                padding= 'same',
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))    
    layer3_1x1 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, 
                                name="layer3_1x1",
                                padding= 'same',
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))    
    layer7_1x1_transp = tf.layers.conv2d_transpose(layer7_1x1, num_classes, 4, 2, 
                                name="layer7_1x1_transp",
                                padding= 'same',
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    layer7_4_skip = tf.add(layer7_1x1_transp, layer4_1x1)  # 1st skip layer
    layer7_4_skip_1x1_transp = tf.layers.conv2d_transpose(layer7_4_skip, num_classes, 4,2 , 
                                name="layer7_4_skip_1x1_transp",
                                padding= 'same',
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    layer7_4_3_skip = tf.add(layer7_4_skip_1x1_transp, layer3_1x1)  # 2nd skip layer    
    layer7_4_3_transpose = tf.layers.conv2d_transpose(layer7_4_3_skip, num_classes, 16, 8, 
                                name="layer7_4_3_transpose",
                                padding= 'same',
                                kernel_initializer= tf.random_normal_initializer(stddev=0.01), 
                                kernel_regularizer= tf.contrib.layers.l2_regularizer(1e-3))
    return layer7_4_3_transpose

#tests.test_layers(layers)

In [6]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """    
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    labels = tf.reshape(correct_label, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)

    return logits, optimizer, cross_entropy_loss

In [7]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
            
    for epoch in range(epochs):
        for batch, (image, label) in enumerate(get_batches_fn(batch_size)):
            #_, loss = sess.run([train_op, cross_entropy_loss], feed_dict=feed_dict)

            feed_dict = {input_image: image, correct_label: label, keep_prob: 0.5, learning_rate: 1e-4}
            _, loss = sess.run([train_op, cross_entropy_loss], feed_dict=feed_dict)
            
            #_, loss = sess.run([train_op, cross_entropy_loss], feed_dict={input_image: image, correct_label: label, keep_prob: KEEP_PROB, learning_rate: LEARNING_RATE})
            print('Epoch ', epoch, ' Batch ', batch, ' Loss ', loss, flush=True)

    pass
            
#tests.test_train_nn(train_nn)

In [8]:
def frame_inference(image, sess, image_shape, logits, keep_prob):
    image = scipy.misc.imresize(image, image_shape)
    tf.Print(keep_prob)
    image_softmax = sess.run(
            [tf.nn.softmax(logits)],
            {keep_prob: keep_prob, image: [image]})
    return image

def video_pipeline(clip, sess, image_shape, logits, keep_prob):
    def image_pipeline(image):
        return frame_inference(image, sess, image_shape, logits, keep_prob)
    return clip.fl_image(image_pipeline)

def video_pipeline_(image, sess, image_shape, logits, keep_prob):
    image = scipy.misc.imresize(image, image_shape)
    
    image_softmax = sess.run(
            [tf.nn.softmax(logits)],
            {keep_prob: keep_prob, image: [image]})
    
    image_softmax = image_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
        
    segmentation = (image_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
    mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
    mask = scipy.misc.toimage(mask, mode="RGBA")
    street_im = scipy.misc.toimage(image)
    street_im.paste(mask, box=None, mask=mask)

    return np.array(street_im)

In [9]:
def process_video(in_video, sess, image_shape, logits, vgg_keep_prob):
    
    in_vid  = 'data/videos/' + in_video
    out_vid = 'runs/videos/'+ in_video
    clip = VideoFileClip(in_vid)
    
    #video_clip = clip.fl_image(video_pipeline)
    video_clip = clip.fx (video_pipeline, sess, image_shape, logits, vgg_keep_prob)
    video_clip.write_videofile(out_vid, audio=False)
    pass

In [10]:
def run():
    global image_shape, sess, logits, keep_prob, input_image

    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)
    train_model = False
    load_model = True

    learning_rate = tf.placeholder(tf.float32)
    correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes])

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    epochs = 40
    batch_size = 8

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # Build NN using load_vgg, layers, and optimize function
        input_image, vgg_keep_prob, vgg_layer3_out, vgg_layer4_out, vgg_layer7_out = load_vgg(sess, vgg_path)
        nn_last_layer = layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes)
        logits, train_op, cross_entropy_loss = optimize(nn_last_layer, correct_label, learning_rate, num_classes)

        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()

        if (train_model):
            
            # Train NN using the train_nn function
            train_nn(sess, epochs, batch_size, get_batches_fn, 
                     train_op, cross_entropy_loss, input_image,
                     correct_label, vgg_keep_prob, learning_rate)
            
            save_path = saver.save(sess, "./saved_network")
            
        elif (load_model):
 
            # Skip the training and load saved trained network instead
            checkpoint = tf.train.get_checkpoint_state("saved_network")
            print("----------------", checkpoint)
            if checkpoint and checkpoint.model_checkpoint_path:
                saver.restore(sess, checkpoint.model_checkpoint_path)
                print("successfuly loaded trained network:", checkpoint.model_checkpoint_path)
            else:
                print("trained network could not be found!")
            

        # Save inference data using helper.save_inference_samples
        #helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, vgg_keep_prob, input_image)

        # Apply the trained model to a video
        in_video = 'challenge_video.mp4'
        process_video(in_video, sess, image_shape, logits, vgg_keep_prob)


In [11]:
if __name__ == '__main__':
    run()

Tests Passed
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
---------------- model_checkpoint_path: "saved_network/net1"
all_model_checkpoint_paths: "saved_network/net1"

INFO:tensorflow:Restoring parameters from saved_network/net1
successfuly loaded trained network: saved_network/net1


TypeError: Print() missing 1 required positional argument: 'data'